**{DEV}\[TODO\]**
- everything

---
### Chapter 1: System Modelling
---
# **Tutorial 1.d: Relative vs Absolute Orientations in 3D**
**Aim:** To compare the performance of simulation when using relative vs absolute orientations in three dimensions

We've already established how to set up the modelling code, and compared relative vs absolute orientations in 2D. Now we will do the same in 3D

## **Contents**:
* [3D Modelling](#3D-Modelling)
* [Spherical Pendulum](#Spherical-Pendulum)
* [Hinge Pendulum](#Hinge-Pendulum)